In [126]:
import random
import torch
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import time

import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
        

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [127]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

In [128]:
SEED = 1038893

fix_random(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

root = "../../data/ml-25m"

Device: cpu


## Data Acquisition

In [129]:
def get_data_from_csv(file: str, nrows=None):
    if nrows:
        df = pd.read_csv(f"{root}/{file}", nrows=nrows)
    else:
        df = pd.read_csv(f"{root}/{file}")
    print(f"Loaded ml-25m data: {root}/{file}")

    return df

def get_ratings_from_csv(nrows=None):
    path = "ratings.csv"

    data = get_data_from_csv(path, nrows)
    # data = get_data_from_csv(path)

    data.drop("timestamp", axis=1, inplace=True)
    # todo: drop user id
    #data.drop("userId", axis=1, inplace=True)

    return data

def get_tag_relevances_from_csv(nrows=None):
    path = "genome-scores.csv"

    return get_data_from_csv(path, nrows)

In [130]:
genome_scores_path = "genome-scores.csv"
ratings_path = "ratings.csv"

In [131]:
ratings = get_ratings_from_csv()
print(ratings)

Loaded ml-25m data: ../../data/ml-25m/ratings.csv
          userId  movieId  rating
0              1      296     5.0
1              1      306     3.5
2              1      307     5.0
3              1      665     5.0
4              1      899     3.5
...          ...      ...     ...
25000090  162541    50872     4.5
25000091  162541    55768     2.5
25000092  162541    56176     2.0
25000093  162541    58559     4.0
25000094  162541    63876     5.0

[25000095 rows x 3 columns]


In [132]:
genome_scores = get_tag_relevances_from_csv()
print(genome_scores)

Loaded ml-25m data: ../../data/ml-25m/genome-scores.csv
          movieId  tagId  relevance
0               1      1    0.02875
1               1      2    0.02375
2               1      3    0.06250
3               1      4    0.07575
4               1      5    0.14075
...           ...    ...        ...
15584443   206499   1124    0.11000
15584444   206499   1125    0.04850
15584445   206499   1126    0.01325
15584446   206499   1127    0.14025
15584447   206499   1128    0.03350

[15584448 rows x 3 columns]


In [133]:
# films = get_data_from_csv(f"{root}/{ratings}")[]


# DATA VISUALIZATION


In [134]:
            # # Fill in missing values with zeros
            # X.fillna(0, inplace=True)


In [135]:
def addColumnOperation(ratings,X):
     # Compute the mean rating for each user
     count_rating = ratings.groupby('movieId', as_index=False)['rating'].count()
     std= ratings.groupby('movieId', as_index=False)['rating'].std()
     std.fillna(0, inplace=True)
     min_ratings= ratings.groupby('movieId', as_index=False)['rating'].min()
     max_ratings= ratings.groupby('movieId', as_index=False)['rating'].max()
     median= ratings.groupby('movieId', as_index=False)['rating'].median()
     operation = pd.DataFrame({'movieId':count_rating['movieId'],'count_rating': count_rating['rating'], 'std': std['rating'], 'min': min_ratings['rating'], 'max': max_ratings['rating'], 'median': median['rating']}) 
     X = pd.merge(X, operation, on='movieId')
     X.drop("movieId", axis=1, inplace=True)
     return X

# FARE TEST CON AVG, STD_DEV

In [140]:
class NeuralNetwork:
    def __init__(self, ratings, relevance, seed=1038893, batch_size=32, hidden_size1=64, hidden_size2=32, lr=0.001):
        # Pivot the relevance DataFrame to create a matrix of tag relevance scores for each movie
        relevance_matrix = relevance.pivot_table(index='movieId', columns='tagId', values='relevance', fill_value=0)

        # Compute the mean rating for each user
        mean_ratings = ratings.groupby('movieId', as_index=False)['rating'].mean()

        # Merge the ratings and relevance data
        X = mean_ratings.merge(relevance_matrix, on='movieId')
        X = addColumnOperation(ratings,X)
        X.columns = X.columns.astype(str)
        
        ratings = None

        #X = X.drop('movieId', axis=1)
        y = X['rating']
        X = X.drop('rating', axis=1)

        # Split the data into training and testing sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        # Convert the data to PyTorch tensors
        self.X_train = torch.tensor(self.X_train.values, dtype=torch.float32)
        self.y_train = torch.tensor(self.y_train.values, dtype=torch.float32)
        self.X_test = torch.tensor(self.X_test.values, dtype=torch.float32)
        self.y_test = torch.tensor(self.y_test.values, dtype=torch.float32)

        # Define the neural network architecture
        self.model = nn.Sequential(
            nn.Linear(X.shape[1], hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, 1)
        )

        # Define the loss function and optimizer
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

        self.batch_size = batch_size

    def train(self, num_epochs):
        # Iterate over the entire training dataset multiple times
        for epoch in range(num_epochs):
            start_time = time.time()
            # Set the model to training mode
            self.model.train()

            # Shuffle the training data
            indices = torch.randperm(len(self.X_train))
            shuffled_X = self.X_train[indices]
            shuffled_y = self.y_train[indices]

            # Divide the training data into batches
            for i in range(0, len(self.X_train), self.batch_size):
                # Zero the gradients
                self.optimizer.zero_grad()

                # Compute the forward pass
                batch_X = shuffled_X[i:i + self.batch_size]
                batch_y = shuffled_y[i:i + self.batch_size]
                outputs = self.model(batch_X)

                # Compute the loss
                loss = self.criterion(outputs.squeeze(), batch_y)

                # Compute the backward pass and update the weights
                loss.backward()
                self.optimizer.step()

            # Evaluate the model on the validation data
            self.model.eval()
            with torch.no_grad():
                val_outputs = self.model(self.X_test)
                val_loss = self.criterion(val_outputs.squeeze(), self.y_test)


            end_time = time.time()
            elapsed_time = end_time - start_time
            # Print the training and validation loss
            print(f'Epoch {epoch + 1}/{num_epochs} - eta: {elapsed_time:.2f}s -\tTrain Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

    def test(self):
    # Set the model to evaluation mode
        self.model.eval()

        # Disable gradient calculation to speed up inference
        with torch.no_grad():
            # Compute the predictions for the test data
            y_pred = self.model(self.X_test).squeeze()

            # Compute the test loss
            test_loss = self.criterion(y_pred, self.y_test)

            # Compute the root mean squared error (RMSE) of the test predictions
            rmse = torch.sqrt(test_loss)

            # Compute the mean squared error (MSE) of the test predictions
            mse = test_loss.item()

            # Compute the mean absolute error (MAE) of the test predictions
            mae = nn.functional.l1_loss(y_pred, self.y_test).item()

            # Compute R^2 score of the test predictions
            ss_res = torch.sum(torch.square(y_pred - self.y_test))
            ss_tot = torch.sum(torch.square(self.y_test - torch.mean(self.y_test)))
            r2 = 1 - ss_res / ss_tot

            # Print the test metrics
            print(f'Test RMSE: {rmse.item():.4f}')
            print(f'Test MSE: {mse:.4f}')
            print(f'Test MAE: {mae:.4f}')
            print(f'Test R^2 score: {r2.item():.4f}')


In [141]:
model = NeuralNetwork(ratings, 
                        genome_scores,
                        seed=1038893, 
                        batch_size=32, 
                        hidden_size1=64, 
                        hidden_size2=32, 
                        lr=0.0005)

In [142]:
model.train(num_epochs=20)

Epoch 1/20 - eta: 0.82s -	Train Loss: 14.9720, Val Loss: 5.1966
Epoch 2/20 - eta: 0.88s -	Train Loss: 0.1828, Val Loss: 0.2158
Epoch 3/20 - eta: 1.09s -	Train Loss: 0.3236, Val Loss: 0.5240
Epoch 4/20 - eta: 0.97s -	Train Loss: 0.2105, Val Loss: 0.2002
Epoch 5/20 - eta: 0.93s -	Train Loss: 0.0851, Val Loss: 0.0813
Epoch 6/20 - eta: 0.94s -	Train Loss: 0.0671, Val Loss: 0.0642
Epoch 7/20 - eta: 0.97s -	Train Loss: 0.0387, Val Loss: 0.0415
Epoch 8/20 - eta: 0.95s -	Train Loss: 0.3128, Val Loss: 2.7968
Epoch 9/20 - eta: 1.02s -	Train Loss: 0.0808, Val Loss: 0.0882
Epoch 10/20 - eta: 0.91s -	Train Loss: 0.0571, Val Loss: 0.0398
Epoch 11/20 - eta: 0.99s -	Train Loss: 0.0158, Val Loss: 0.0290
Epoch 12/20 - eta: 0.97s -	Train Loss: 0.0206, Val Loss: 0.0325
Epoch 13/20 - eta: 1.26s -	Train Loss: 0.0325, Val Loss: 0.0229
Epoch 14/20 - eta: 1.20s -	Train Loss: 0.0300, Val Loss: 0.0822
Epoch 15/20 - eta: 1.02s -	Train Loss: 0.0270, Val Loss: 0.1155
Epoch 16/20 - eta: 0.98s -	Train Loss: 0.9708, V

In [143]:
model.test()

Test RMSE: 0.1356
Test MSE: 0.0184
Test MAE: 0.1005
Test R^2 score: 0.9201
